In [19]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right

In [189]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning

TRESH_DIST = 10 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
#             cv2.imshow('running frame', frame)

                
            ##### APPELER get_vision_position##########
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            
            #if no
            ######APPELER PID##########
            print()
            sum_error, alt_error_pid, v_l,v_d = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt, True)
#             v_l = 50
#             v_d = 50
            v = (v_l + v_d)*SPEED_TO_MMS/2
            yaw = (v_d - v_l)*YAW_TO_MMS
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
#             u_t_minus_1 = 0;
#             set_speed(int(v_l), int(v_d))

    
    
            #if yes, obstacle avoidance 
            
            actual = time.time()
            diff = actual-previous
            print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        cv2.destroyAllWindows()



In [124]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [125]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [10]:
import tdmclient.notebook
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [186]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)

<module 'computer_vision' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\computer_vision.py'>

In [ ]:
main()

take the rigth image
Px_to_mm=  0.4761494257045783


100%|██████████| 2/2 [00:00<00:00, 517.78it/s]


goal is:  [271.36 103.04]

pos robot is:  [259.    107.32   55.463]
y: -4.279523809523809
x: 12.36047619047622
Robot_angle: 304.53678210332055
angle goal:  340.9027932294573
Error_angle: 645.4395753327778
Turning only, speed l and r: 322.7197876663889 -322.7197876663889
Sum_error, Alt_error_pid: 50 0
elapsed time is:  0.11928200721740723
goal is:  [271.36 103.04]

pos robot is:  [259.088 107.408  54.004]
y: -4.368113207547168
x: 12.271886792452847
Robot_angle: 305.9960022809168
angle goal:  340.4070820177797
Error_angle: 646.4030842986965
Turning only, speed l and r: 323.20154214934826 -323.20154214934826
Sum_error, Alt_error_pid: 50 645.4395753327778
elapsed time is:  0.05603289604187012
goal is:  [271.36 103.04]

pos robot is:  [259.104 107.818  53.191]
y: -4.777971014492749
x: 12.256231884057968
Robot_angle: 306.80852162898304
angle goal:  338.70216747069554
Error_angle: 645.5106890996785
Turning only, speed l and r: 322.75534454983926 -322.75534454983926
Sum_error, Alt_error_pid: 5

goal is:  [271.36 103.04]
Robot not seen by camera3

pos robot is:  [260.907 110.169  48.523]
y: -7.128969157742105
x: 10.453172011013066
Robot_angle: 311.4773846743395
angle goal:  325.7063644005363
Error_angle: 637.1837490748758
Turning only, speed l and r: 318.5918745374379 -318.5918745374379
Sum_error, Alt_error_pid: 50 640.4270446510237
elapsed time is:  0.05400681495666504
goal is:  [271.36 103.04]

pos robot is:  [255.049 112.43   43.86 ]
y: -9.390395590344312
x: 16.311388875518134
Robot_angle: 316.1397426591391
angle goal:  330.07115304581936
Error_angle: 646.2108957049584
Turning only, speed l and r: 323.1054478524792 -323.1054478524792
Sum_error, Alt_error_pid: 50 637.1837490748758
elapsed time is:  0.07290768623352051
goal is:  [271.36 103.04]

pos robot is:  [258.744 111.712  49.39 ]
y: -8.6720166133412
x: 12.615902312292235
Robot_angle: 310.61016707649856
angle goal:  325.49584476827295
Error_angle: 636.1060118447715
Turning only, speed l and r: 318.05300592238575 -318.053


pos robot is:  [259.102 107.517  53.128]
y: -76.79739803288551
x: 195.93825479066595
Robot_angle: 306.87156103890993
angle goal:  338.5974561439653
Error_angle: 645.4690171828752
Turning only, speed l and r: 322.7345085914376 -322.7345085914376
Sum_error, Alt_error_pid: 50 645.4814269269823
elapsed time is:  0.0826570987701416
goal is:  [455.04  30.72]

pos robot is:  [259.106 107.462  53.147]
y: -76.74216801623756
x: 195.93436811123922
Robot_angle: 306.853182681328
angle goal:  338.6110711171633
Error_angle: 645.4642537984913
Turning only, speed l and r: 322.73212689924566 -322.73212689924566
Sum_error, Alt_error_pid: 50 645.4690171828752
elapsed time is:  0.08397674560546875
goal is:  [455.04  30.72]

pos robot is:  [259.107 107.441  53.154]
y: -76.72107202707726
x: 195.93288353180156
Robot_angle: 306.8461517002107
angle goal:  338.6162723858189
Error_angle: 645.4624240860296
Turning only, speed l and r: 322.7312120430148 -322.7312120430148
Sum_error, Alt_error_pid: 50 645.464253798

...


pos robot is:  [259.112 107.207  53.186]
y: -76.4870523171804
x: 195.92751162200034
Robot_angle: 306.8141009796233
angle goal:  338.6750999487213
Error_angle: 645.4892009283446
Turning only, speed l and r: 322.7446004641723 -322.7446004641723
Sum_error, Alt_error_pid: 50 645.5340261568374
elapsed time is:  0.09517216682434082
goal is:  [455.04  30.72]

pos robot is:  [259.11  106.948  53.169]
y: -76.22808474830119
x: 195.9302645859558
Robot_angle: 306.83128181172196
angle goal:  338.74111712378436
Error_angle: 645.5723989355063
Turning only, speed l and r: 322.78619946775314 -322.78619946775314
Sum_error, Alt_error_pid: 50 645.4892009283446
elapsed time is:  0.12531375885009766
goal is:  [455.04  30.72]

pos robot is:  [259.109 107.245  52.882]
y: -76.52470969177318
x: 195.931316124617
Robot_angle: 307.11843363157885
angle goal:  338.66592163806234
Error_angle: 645.7843552696412
Turning only, speed l and r: 322.8921776348206 -322.8921776348206
Sum_error, Alt_error_pid: 50 645.57239893


pos robot is:  [259.108 107.195  52.888]
y: -76.4754612524671
x: 195.93196600672428
Robot_angle: 307.1122329921197
angle goal:  338.67848257347936
Error_angle: 645.790715565599
Turning only, speed l and r: 322.8953577827995 -322.8953577827995
Sum_error, Alt_error_pid: 50 645.5889869997957
elapsed time is:  0.13312602043151855
goal is:  [455.04  30.72]

pos robot is:  [259.108 106.944  53.328]
y: -76.22365735554651
x: 195.93196600952137
Robot_angle: 306.6716663005887
angle goal:  338.74240974037014
Error_angle: 645.4140760409589
Turning only, speed l and r: 322.70703802047944 -322.70703802047944
Sum_error, Alt_error_pid: 50 645.790715565599
elapsed time is:  0.15521979331970215
goal is:  [455.04  30.72]

pos robot is:  [259.108 106.847  53.223]
y: -76.12747682542252
x: 195.9319660105898
Robot_angle: 306.77670196680276
angle goal:  338.7668424099806
Error_angle: 645.5435443767833
Turning only, speed l and r: 322.77177218839165 -322.77177218839165
Sum_error, Alt_error_pid: 50 645.4140760


pos robot is:  [264.671 103.951  75.806]
y: -73.23146390452833
x: 190.3689144471938
Robot_angle: 284.1940250597796
angle goal:  338.95917220778585
Error_angle: 623.1531972675655
Turning only, speed l and r: 311.57659863378274 -311.57659863378274
Sum_error, Alt_error_pid: 50 601.8746140017347
elapsed time is:  0.09679746627807617
goal is:  [455.04  30.72]

pos robot is:  [262.024  97.794  70.409]
y: -67.07372557204975
x: 193.015985889349
Robot_angle: 289.59137834982306
angle goal:  340.83748306233855
Error_angle: 630.4288614121616
Turning only, speed l and r: 315.2144307060808 -315.2144307060808
Sum_error, Alt_error_pid: 50 623.1531972675655
elapsed time is:  0.1337604522705078
goal is:  [455.04  30.72]

pos robot is:  [269.319  93.464  79.496]
y: -62.74397005887138
x: 185.7207004008045
Robot_angle: 280.5037524266641
angle goal:  341.3329499906875
Error_angle: 621.8367024173516
Turning only, speed l and r: 310.9183512086758 -310.9183512086758
Sum_error, Alt_error_pid: 50 630.4288614121

elapsed time is:  0.15707707405090332
goal is:  [455.04  30.72]

pos robot is:  [290.888 140.062 225.602]
y: -109.34194742186469
x: 164.15225644399771
Robot_angle: 134.3979916612178
angle goal:  326.33238464210024
Error_angle: 460.73037630331805
Turning only, speed l and r: 230.36518815165903 -230.36518815165903
Sum_error, Alt_error_pid: 50 437.13903197608425
elapsed time is:  0.10158991813659668
goal is:  [455.04  30.72]
Robot not seen by camera3

pos robot is:  [291.888 141.062 223.798]
y: -110.34194742186469
x: 163.15225644399771
Robot_angle: 136.2019967313998
angle goal:  325.9290103904512
Error_angle: 462.131007121851
Turning only, speed l and r: 231.0655035609255 -231.0655035609255
Sum_error, Alt_error_pid: 50 460.73037630331805
elapsed time is:  0.088592529296875
goal is:  [455.04  30.72]

pos robot is:  [288.637 145.985 219.184]
y: -115.26525967752372
x: 166.40293572751273
Robot_angle: 140.81596325264545
angle goal:  325.29013231086043
Error_angle: 466.10609556350585
Turning on


pos robot is:  [259.317 107.406  53.33 ]
y: -76.68600787320614
x: 195.7231707320467
Robot_angle: 306.6704392888707
angle goal:  338.6043303738597
Error_angle: 645.2747696627304
Turning only, speed l and r: 322.6373848313652 -322.6373848313652
Sum_error, Alt_error_pid: 50 644.9758781038265
elapsed time is:  0.10647988319396973
goal is:  [455.04  30.72]

pos robot is:  [259.188 107.42   53.224]
y: -76.69962076125232
x: 195.8522133112849
Robot_angle: 306.77590945213535
angle goal:  338.61370123350156
Error_angle: 645.3896106856369
Turning only, speed l and r: 322.6948053428184 -322.6948053428184
Sum_error, Alt_error_pid: 50 645.2747696627304
elapsed time is:  0.1329793930053711
goal is:  [455.04  30.72]

pos robot is:  [259.138 107.425  52.903]
y: -76.7048204218009
x: 195.901503190558
Robot_angle: 307.0968583544365
angle goal:  338.6172776415529
Error_angle: 645.7141359959894
Turning only, speed l and r: 322.8570679979947 -322.8570679979947
Sum_error, Alt_error_pid: 50 645.3896106856369


In [123]:
cap.release()

In [188]:
set_speed(0, 0)